In [1]:
#import packages
import pandas as pd
import sys
import matplotlib.pyplot as plt
from datetime import datetime
import logging

#custom classes
from classes import classes as cl
from functions import functions as fc
from functions import routing as rt

In [2]:
#create empty dict
initiation_objects = {'dict_depots': '', 'dict_sites': '', 'dict_plants': '', 'dict_jobs': '', 'dict_tours': '', 'list_days': ''}
init_path = 'output/inital_solution'

objects = fc.initiate(initiation_objects,init_path,'final')

Starting initiation at 22:49:18
{'Neuherberg': <classes.classes.Depot object at 0x0000014D9C71EA30>, 'Lauterhofen': <classes.classes.Depot object at 0x0000014DA0117E20>, 'Altbach': <classes.classes.Depot object at 0x0000014DA0117E80>, 'Deisslingen': <classes.classes.Depot object at 0x0000014DA0117EE0>, 'Bollschweil': <classes.classes.Depot object at 0x0000014DA0117F40>, 'Lampertheim': <classes.classes.Depot object at 0x0000014DA0117FA0>, 'Neuss': <classes.classes.Depot object at 0x0000014D9C7181C0>, 'Embsen': <classes.classes.Depot object at 0x0000014D9C7180D0>, 'Rottleberode': <classes.classes.Depot object at 0x0000014D9C7184C0>, 'Fenne': <classes.classes.Depot object at 0x0000014D9C718310>, 'Castrop': <classes.classes.Depot object at 0x0000014D9C7183D0>, 'Perl': <classes.classes.Depot object at 0x0000014D9C718550>, 'Gnetsch': <classes.classes.Depot object at 0x0000014D9C726550>, 'Vogelsdorf': <classes.classes.Depot object at 0x0000014D9C7262B0>, 'Sukow': <classes.classes.Depot object

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [3]:
list_days = objects['list_days']

In [4]:
list_days

NameError: name 'list_days' is not defined